In [1]:
from Pyfhel import Pyfhel, PyCtxt, PyPtxt


In [2]:
HE = Pyfhel()
HE.contextGen(p=2**16 +1 , m=2**12)
HE.keyGen()

In [3]:
import base64

ca = HE.encryptFrac(5.0)
ca_bytes = PyCtxt.to_bytes(ca)
ca_bytes_base64 = base64.b64encode(ca_bytes) 
ca_string = ca_bytes_base64.decode('utf-8')
#str(ca_string)print(str(ca_string))


In [4]:
from influxdb import InfluxDBClient
import base64
json_body = [
    {
        "measurement": "testMeas",
        "tags": {
            "device": "001",
            "location": "Messina"
        },
        "fields": {
            "value": ca_string
        }
    }
]

client = InfluxDBClient(host='influxdb', port=8086, username='admin', password='temp', database='security_db', ssl=False)
client.write_points(json_body)
result = client.query('select value from testMeas;')
#print("Result: {0}".format(result))




In [5]:
results = client.query('SELECT value FROM testMeas')
#print(results.raw)
results.get_points()
points = results.get_points()
ca_restored = PyCtxt()

for point in points:
    print(type(point['value']))
    ca_string_restored = point['value']
    assert(ca_string_restored == ca_string)
    print('Worked !')

    ca_bytes_base64_restored = ca_string_restored.encode('utf-8')
    assert(ca_bytes_base64_restored == ca_bytes_base64)
    print('Worked !')

    ca_bytes_restored = base64.b64decode(ca_bytes_base64_restored)
    assert(ca_bytes_restored == ca_bytes) 
    print('Worked !')

    ca_restored.from_bytes(ca_bytes_restored, float)
    print(HE.decryptFrac(ca_restored))

<class 'str'>
Worked !
Worked !
Worked !
5.0


In [6]:
client.drop_measurement('testMeas')